In [2]:
import numpy as np
import pandas as pd
import pytesseract
import cv2
import spacy
import re
import string

In [3]:
def cleanText(txt):
  whitespace=string.whitespace
  punctuation = '!#$%&\'()*+:;<=>?[\\]^`{|}~'

  tableWhitespace = str.maketrans('','',whitespace)
  tablePunctuation = str.maketrans('','',punctuation)
  text = str(txt)
  text = text.lower()
  removewhitespace = text.translate(tableWhitespace)
  removepunctuation = removewhitespace.translate(tablePunctuation)

  return str(removepunctuation)

In [4]:
# Load NER model
model_ner = spacy.load(r'C:\Users\manab\OneDrive\Desktop\Project\Train\output\model-best')

In [5]:
# Load Image
image = cv2.imread(r'C:\Users\manab\OneDrive\Desktop\Project\Selected\060.jpeg')
image = cv2.resize(image,(640,360))
# cv2.imshow('business Card', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# extract data using Spytesseract
tessdata = pytesseract.image_to_data(image)
# convert data into content
tesslist= list(map(lambda x: x.split('\t'),tessdata.split('\n')))

df = pd.DataFrame(tesslist[1:],columns=tesslist[0])
df.dropna(inplace=True)
df['text']=df['text'].apply(cleanText)
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)
# get prediction from NER model

doc = model_ner(content)

y university of south australia dr jonathan smithiskywich pp, msc, 85¢ tions senior lecturer 618 8302 0000 school of nursing and midwifery m 61 422 333 555 division of health sciences jonathan.smithiskywich@unisa.edu.au ‘wow.unisa.edu.auftee/cangoon/forever gpo box 2471, adelaide, south australia 5001


In [6]:
from spacy import displacy

In [7]:
displacy.serve(doc, style='ent')

c:\python\Python312\Lib\site-packages\spacy\displacy\__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [15]:
displacy.render(doc, style='ent')

Tagging

In [9]:
doc_json = doc.to_json()
doc_json.keys()

dict_keys(['text', 'ents', 'tokens'])

In [10]:
doc_text = doc_json['text']

In [11]:
datafram_tokens = pd.DataFrame(doc_json['tokens'])
datafram_tokens['tokens'] = datafram_tokens[['start','end']].apply(lambda x:doc_text[x[0]:x[1]], axis = 1)

datafram_tokens.head(10)

C:\Users\manab\AppData\Local\Temp\ipykernel_4876\752382242.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datafram_tokens['tokens'] = datafram_tokens[['start','end']].apply(lambda x:doc_text[x[0]:x[1]], axis = 1)


,id,start,end,tokens
0,0,0,1,y
1,1,2,12,university
2,2,13,15,of
3,3,16,21,south
4,4,22,31,australia
5,5,32,34,dr
6,6,35,43,jonathan
7,7,44,57,smithiskywich
8,8,58,60,pp
9,9,60,61,","


In [12]:
righ_table = pd.DataFrame(doc_json['ents'])[['start', 'label']]
datafram_tokens = pd.merge(datafram_tokens,righ_table,how='left',on='start')

In [13]:
datafram_tokens.fillna('O', inplace=True)
datafram_tokens.head(10)

,id,start,end,tokens,label
0,0,0,1,y,B-ORG
1,1,2,12,university,I-ORG
2,2,13,15,of,I-ORG
3,3,16,21,south,I-ORG
4,4,22,31,australia,I-ORG
5,5,32,34,dr,B-NAME
6,6,35,43,jonathan,I-NAME
7,7,44,57,smithiskywich,I-NAME
8,8,58,60,pp,O
9,9,60,61,",",O


In [14]:
df_clean.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
8,5,1,2,1,1,1,83,36,34,40,26,y
12,5,1,3,1,1,1,52,94,74,14,96,university
13,5,1,3,1,1,2,131,93,16,12,95,of
15,5,1,3,1,2,1,40,111,43,12,96,south
16,5,1,3,1,2,2,89,111,67,12,96,australia


In [15]:
# join Lable to df_clean dataframe
df_clean['end']=df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1
df_clean['start']=df_clean[['text','end']].apply(lambda x: x[1]-len(x[0]), axis=1)

C:\Users\manab\AppData\Local\Temp\ipykernel_4876\3062264408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['end']=df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1
C:\Users\manab\AppData\Local\Temp\ipykernel_4876\3062264408.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_clean['start']=df_clean[['text','end']].apply(lambda x: x[1]-len(x[0]), axis=1)
C:\Users\manab\AppData\Local\Temp\ipykernel_4876\3062264408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [16]:
df_clean

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start
8,5,1,2,1,1,1,83,36,34,40,26,y,1,0
12,5,1,3,1,1,1,52,94,74,14,96,university,12,2
13,5,1,3,1,1,2,131,93,16,12,95,of,15,13
15,5,1,3,1,2,1,40,111,43,12,96,south,21,16
16,5,1,3,1,2,2,89,111,67,12,96,australia,31,22
20,5,1,4,1,1,1,42,176,22,13,95,dr,34,32
21,5,1,4,1,1,2,70,175,91,15,92,jonathan,43,35
22,5,1,4,1,1,3,169,175,142,19,91,smithiskywich,57,44
23,5,1,4,1,1,4,318,181,24,10,18,"pp,",61,58
24,5,1,4,1,1,5,348,181,26,10,85,"msc,",66,62


In [17]:
# inner join with start
dataframe_info = pd.merge(df_clean, datafram_tokens[['start','tokens', 'label']], how = 'inner', on='start')

In [18]:
dataframe_info

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label
0,5,1,2,1,1,1,83,36,34,40,26,y,1,0,y,B-ORG
1,5,1,3,1,1,1,52,94,74,14,96,university,12,2,university,I-ORG
2,5,1,3,1,1,2,131,93,16,12,95,of,15,13,of,I-ORG
3,5,1,3,1,2,1,40,111,43,12,96,south,21,16,south,I-ORG
4,5,1,3,1,2,2,89,111,67,12,96,australia,31,22,australia,I-ORG
5,5,1,4,1,1,1,42,176,22,13,95,dr,34,32,dr,B-NAME
6,5,1,4,1,1,2,70,175,91,15,92,jonathan,43,35,jonathan,I-NAME
7,5,1,4,1,1,3,169,175,142,19,91,smithiskywich,57,44,smithiskywich,I-NAME
8,5,1,4,1,1,4,318,181,24,10,18,"pp,",61,58,pp,O
9,5,1,4,1,1,5,348,181,26,10,85,"msc,",66,62,msc,O


Bounding Box

In [57]:
# Filter the dataframe to only get labeled bounding boxes
boundingbox_dataframe = dataframe_info.query("label != 'O'")
img = image.copy()

for x, y, w, h, label in boundingbox_dataframe[['left', 'top', 'width', 'height', 'label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)

    # Corrected rectangle drawing and text placement
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.putText(img, str(label), (x, y-5), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)

cv2.imshow('Prediction', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [58]:
boundingbox_dataframe['label']=boundingbox_dataframe['label'].apply(lambda x: x[2:])
boundingbox_dataframe

C:\Users\manab\AppData\Local\Temp\ipykernel_12388\2975471317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boundingbox_dataframe['label']=boundingbox_dataframe['label'].apply(lambda x: x[2:])


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label
0,5,1,2,1,1,1,83,36,34,40,26,y,1,0,y,ORG
1,5,1,3,1,1,1,52,94,74,14,96,university,12,2,university,ORG
2,5,1,3,1,1,2,131,93,16,12,95,of,15,13,of,ORG
3,5,1,3,1,2,1,40,111,43,12,96,south,21,16,south,ORG
4,5,1,3,1,2,2,89,111,67,12,96,australia,31,22,australia,ORG
5,5,1,4,1,1,1,42,176,22,13,95,dr,34,32,dr,NAME
6,5,1,4,1,1,2,70,175,91,15,92,jonathan,43,35,jonathan,NAME
7,5,1,4,1,1,3,169,175,142,19,91,smithiskywich,57,44,smithiskywich,NAME
12,5,1,5,1,1,1,41,211,40,11,95,senior,83,77,senior,DES
23,5,1,5,1,2,7,476,228,20,9,92,61,143,141,61,PHONE


In [59]:
class groupgen():
  def __init__(self):
    self.id = 0
    self.text = ''

  def getgroup(self,text):
     if self.text == text:
       return self.id
     else:
       self.id +=1
       self.text = text
       return self.id

grp_gen = groupgen()     

In [60]:
boundingbox_dataframe['group'] = boundingbox_dataframe['label'].apply(grp_gen.getgroup)
boundingbox_dataframe

C:\Users\manab\AppData\Local\Temp\ipykernel_12388\4136687567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boundingbox_dataframe['group'] = boundingbox_dataframe['label'].apply(grp_gen.getgroup)


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label,group
0,5,1,2,1,1,1,83,36,34,40,26,y,1,0,y,ORG,1
1,5,1,3,1,1,1,52,94,74,14,96,university,12,2,university,ORG,1
2,5,1,3,1,1,2,131,93,16,12,95,of,15,13,of,ORG,1
3,5,1,3,1,2,1,40,111,43,12,96,south,21,16,south,ORG,1
4,5,1,3,1,2,2,89,111,67,12,96,australia,31,22,australia,ORG,1
5,5,1,4,1,1,1,42,176,22,13,95,dr,34,32,dr,NAME,2
6,5,1,4,1,1,2,70,175,91,15,92,jonathan,43,35,jonathan,NAME,2
7,5,1,4,1,1,3,169,175,142,19,91,smithiskywich,57,44,smithiskywich,NAME,2
12,5,1,5,1,1,1,41,211,40,11,95,senior,83,77,senior,DES,3
23,5,1,5,1,2,7,476,228,20,9,92,61,143,141,61,PHONE,4


In [61]:
# right and bottom of bounding box
boundingbox_dataframe[['left','top','width','height']]=boundingbox_dataframe[['left','top','width','height']].astype(int)
boundingbox_dataframe['right']= boundingbox_dataframe['left']+boundingbox_dataframe['width']
boundingbox_dataframe['bottom']= boundingbox_dataframe['top']+boundingbox_dataframe['height']

C:\Users\manab\AppData\Local\Temp\ipykernel_12388\1891877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boundingbox_dataframe[['left','top','width','height']]=boundingbox_dataframe[['left','top','width','height']].astype(int)
C:\Users\manab\AppData\Local\Temp\ipykernel_12388\1891877.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boundingbox_dataframe['right']= boundingbox_dataframe['left']+boundingbox_dataframe['width']
C:\Users\manab\AppData\Local\Temp\ipykernel_12388\1891877.py:4: SettingWithCo

In [34]:
boundingbox_dataframe

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label,group,right,bottom
0,5,1,2,1,1,1,83,36,34,40,26,y,1,0,y,ORG,1,117,76
1,5,1,3,1,1,1,52,94,74,14,96,university,12,2,university,ORG,1,126,108
2,5,1,3,1,1,2,131,93,16,12,95,of,15,13,of,ORG,1,147,105
3,5,1,3,1,2,1,40,111,43,12,96,south,21,16,south,ORG,1,83,123
4,5,1,3,1,2,2,89,111,67,12,96,australia,31,22,australia,ORG,1,156,123
5,5,1,4,1,1,1,42,176,22,13,95,dr,34,32,dr,NAME,2,64,189
6,5,1,4,1,1,2,70,175,91,15,92,jonathan,43,35,jonathan,NAME,2,161,190
7,5,1,4,1,1,3,169,175,142,19,91,smithiskywich,57,44,smithiskywich,NAME,2,311,194
12,5,1,5,1,1,1,41,211,40,11,95,senior,83,77,senior,DES,3,81,222
23,5,1,5,1,2,7,476,228,20,9,92,61,143,141,61,PHONE,4,496,237


Tagging

In [36]:
# tagging : group by group
col_group = ['left', 'top', 'right', 'bottom', 'label', 'tokens', 'group']
group_tag_img = boundingbox_dataframe[col_group].groupby(by='group')

In [37]:
import numpy as np
img_tagging=group_tag_img.agg({
  'left':min,
  'right':max,
  'top':min,
  'bottom':max,
  'label':np.unique,
  'tokens':lambda x: " ".join(x)
})

C:\Users\manab\AppData\Local\Temp\ipykernel_12388\2820685258.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  img_tagging=group_tag_img.agg({
C:\Users\manab\AppData\Local\Temp\ipykernel_12388\2820685258.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  img_tagging=group_tag_img.agg({


In [38]:
img_tagging

,left,right,top,bottom,label,tokens
group,,,,,,
1,40,156,36,123,[ORG],y university of south australia
2,42,311,175,194,[NAME],dr jonathan smithiskywich
3,41,81,211,222,[DES],senior
4,476,551,228,237,[PHONE],61 422 333
5,359,577,244,256,[EMAIL],jonathan.smithiskywich@unisa.edu.au
6,347,577,260,273,[WEB],‘


In [50]:
image_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(image_bb,(l,t),(r,b),(0,255,0),2)
    cv2.putText(image_bb,str(label[0]),(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255))

cv2.imshow('Bounding Box BusinessCard', image_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()   

Parser

In [52]:
import re
def parser(text, label):
  if label == 'PHONE':
    text = text.lower()
    text = re.sub(r'\D','',text)
  elif label == 'EMAIL':
    text = text.lower()
    allow_spcial_char = r'@_.\-'
    text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_spcial_char),'',text)
  elif label == 'WEB':
    text = text.lower()
    allow_spcial_char = r':/.%#\-'
    text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_spcial_char),'',text)
  elif label in ('NAME', 'DES'):
    text =  text.lower()
    text = re.sub(r'[^a-z ]','',text)
    text = text.title()
  elif label == 'ORG':
    text =  text.lower()
    text = re.sub(r'[^a-z0-9 ]','',text)
    text = text.title()

  return text  


In [53]:
parser('Manab 1114938973*&(@*&@*E(*@~)*#())', 'NAME')

'Manab E'

Entities

In [63]:
info_array=dataframe_info[['tokens','label']].values
entities = dict(NAME=[],
                ORG=[],
                DES=[],
                PHONE=[],
                EMAIL=[],
                WEB=[])
previous ='O'
for token, label in info_array:
  bio_tag = label[0]
  label_tag = label[2:]
  
  text = parser(token,label_tag)
  
  if bio_tag in ('B', 'I'):
    if previous != label_tag:
      entities[label_tag].append(text)
    else:  
        if bio_tag == "B":
           entities[label_tag].append(text)
        else:
           if label_tag in ("NAME", "ORG", "DES"):
              entities[label_tag][-1] = entities[label_tag][-1]+" "+text
           else :
              entities[label_tag][-1] = entities[label_tag][-1]+text  
    previous = label_tag
              

In [64]:
entities

{'NAME': ['Dr Jonathan Smithiskywich'],
 'ORG': ['Y University Of South Australia'],
 'DES': ['Senior'],
 'PHONE': ['61422333'],
 'EMAIL': ['jonathan.smithiskywich@unisa.edu.au'],
 'WEB': ['']}